# Curio Bot Agent

### A tiny, creative MCP server that exposes a few real‑world tools (weather, news, wiki) and lets an external LLM act as a router/orchestrator.

For weather, it uses free [Open Meteo API](https://open.meteo.com/).
For News, it uses [Newsapi API](https://newsapi.org/).

You need to get your free API key from the [Newsapi API](https://newsapi.org/) to try it out

Save the generated key as NEWSAPI_KEY to your .env file


In [1]:
from IPython.display import display, Markdown
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from dotenv import load_dotenv
import sys, os, pathlib

load_dotenv(override=True)

True

In [2]:
PROJECT_ROOT = pathlib.Path(os.getcwd())
child_env = dict(os.environ)
child_env.setdefault("PYTHONUNBUFFERED", "1")
child_env["PYTHONPATH"] = str(PROJECT_ROOT) + (":" + child_env["PYTHONPATH"] if "PYTHONPATH" in child_env else "")

child_env["LOG_LEVEL"] = "DEBUG"          # INFO/DEBUG/WARN/ERROR
child_env["LOG_JSON"]  = "false"          # "true" for JSON lines
child_env["LOG_FILE"]  = str(PROJECT_ROOT / "logs" / "curiobot.log")  # optional file sink
child_env["OPENAI_LOG"]    = "debug"      # OpenAI SDK (if used)

params = {"command": sys.executable, "args": ["-m", "server.curiobot_server"], "env":child_env}

async with MCPServerStdio(params=params, client_session_timeout_seconds=120) as server:
    tools = await server.list_tools()
tools

[Tool(name='get_weather', description="Weather via Open-Meteo for a location. 'when' accepts 'today'/'tomorrow'.", inputSchema={'properties': {'location': {'title': 'Location', 'type': 'string'}, 'when': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'When'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_news', description='Topical news via NewsAPI. Requires NEWSAPI_KEY env var.', inputSchema={'properties': {'query': {'title': 'Query', 'type': 'string'}, 'freshness_days': {'default': 3, 'title': 'Freshness Days', 'type': 'integer'}}, 'required': ['query'], 'title': 'get_newsArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_wiki', description='Wikipedia summary for a topic.', inputSchema={'properties': {'topic': {'title': 'Topic', 'type': 'string'}}, 'required': ['topic'], 'title': 'get_wikiArguments', 'type': 'object'}, annotations=None),
 Tool(name='query', description='Nat

In [4]:
# Demonstrate LLM routing by explicitly invoking the 'query' tool and allowing the model to decide which internal tool to use.
PROJECT_ROOT = pathlib.Path(os.getcwd())
child_env = dict(os.environ)
child_env.setdefault("PYTHONUNBUFFERED", "1")
child_env["PYTHONPATH"] = str(PROJECT_ROOT) + (":" + child_env["PYTHONPATH"] if "PYTHONPATH" in child_env else "")

child_env["LOG_LEVEL"] = "DEBUG"          # INFO/DEBUG/WARN/ERROR
child_env["LOG_JSON"]  = "false"          # "true" for JSON lines
child_env["LOG_FILE"]  = str(PROJECT_ROOT / "logs" / "curiobot.log")  # optional file sink
child_env["OPENAI_LOG"]    = "debug"      # OpenAI SDK (if used)

provider = os.getenv("MODEL_PROVIDER", "openai").lower()
child_env["MODEL_PROVIDER"] = provider

# Choose model based on provider
if provider == "openai":
    model = os.getenv("OPENAI_MODEL", "gpt-4.1-mini")
    child_env["OPENAI_MODEL"] = model
else:
    raise ValueError(f"Unknown MODEL_PROVIDER: {provider}")

params = {"command": sys.executable, "args": ["-m", "server.curiobot_server"], "env":child_env}

instructions = """
You are able to answer questions about the actual weather of a given city. When a user asks about the weather, try to give a detailed answer.
CRITICAL: never call get_weather, get_news, or get_wiki directly.
Always call the single router tool `query` with the user's question. 
"""

request = "What's the weather like in Sydney tomorrow?"


async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    agent = Agent(name="weather_specialist", instructions=instructions, model=model, mcp_servers=[server])
    with trace("weather_specialist"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The weather forecast for Sydney tomorrow is looking warm with temperatures reaching up to 30°C. Here's a detailed breakdown:

- **Morning (00:00 - 12:00)**: 
  - Early morning temperatures will start around 17°C, gradually rising to 24°C by late morning.
  - Low probability of precipitation (around 0%-3%).

- **Afternoon (12:00 - 18:00)**: 
  - Peak temperatures will hit around 30°C at 14:00.
  - Slightly higher chance of rain, especially during midday to early afternoon (up to 58% probability).

- **Evening (18:00 - 23:00)**: 
  - Temperatures will drop to about 23°C by late evening.
  - The probability of rain decreases but remains present (around 18%-30%).

Overall, expect a warm day with a chance of rain, especially in the afternoon.

In [5]:
instructions = """
You are a very helpful news specilaist. 
You are able to fetch latest news about a given subject. 
Please choose the latest news about the subject and get all the details.
Please strictly use anthropic provider. Do not proceed if provider is not anthropic and return informative message.
"""
request = "What's the breaking news in Sydney ?"
model = "gpt-4.1-mini"  

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    agent = Agent(name="news_specialist", instructions=instructions, model=model, mcp_servers=[server])
    with trace("news_specialist"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The latest breaking news in Sydney includes a financial outlook update at the Sohn Australia conference:

- Dan Loeb of Third Point LLC expressed confidence in the US markets and isn't concerned about the Federal Reserve's interest rate path. This was shared at the annual Sohn Australia conference held in Sydney on Friday.
- Also at the conference, discussions included topics on China, with other notable speakers contributing insights.

For more details, you can read the article from Financial Post titled "Loeb Upbeat on US Markets, Hintze Talks China at Sohn Australia": https://financialpost.com/pmn/business-pmn/loeb-upbeat-on-us-markets-hintze-talks-china-at-sohn-australia

If you want news focusing more specifically on Sydney local events or other domains, please let me know!

In [6]:
instructions = """
You are a very helpful Wiki Specialist.
You are able to fetch the latest Wikipedia information about a given subject.
Please provide the most up-to-date and accurate summary available from Wikipedia for the requested topic.
If you cannot find current data, clearly say so and suggest related areas to check.
"""
topic = "Anthropic"
model = "gpt-4.1-mini"  

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    agent = Agent(name="wiki_specialist", instructions=instructions, model=model, mcp_servers=[server])
    with trace("wiki_specialist"):
        result = await Runner.run(agent, topic)
    display(Markdown(result.final_output))

I am currently unable to access the latest Wikipedia summary for "Anthropic" due to a technical restriction. However, I can provide a brief overview based on what I know:

Anthropic typically refers to concepts related to humans or human beings. It is often used in contexts such as the "anthropic principle" in cosmology and philosophy, which considers the universe's laws and constants as they relate to the existence of human life.

If you are referring to a specific entity named Anthropic, such as a company or organization, please let me know, and I can provide more detailed information based on available knowledge up to now.

Would you like me to provide information on the anthropic principle, or details about any particular usage of "Anthropic"?

In [7]:
instructions = "You are able to answer questions about the actual weather of a given city. When a user asks about the weather, try to give a detailed answer."
request = "What's the weather like in Sydney tomorrow?"
model = "gpt-4.1-mini"  

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    agent = Agent(name="weather_specialist", instructions=instructions, model=model, mcp_servers=[server])
    with trace("weather_specialist"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Tomorrow in Sydney, the weather will start off cool with temperatures around 17.1°C at midnight and gradually warming up through the morning. By late morning and early afternoon, temperatures will rise to around 30.1°C to 30.5°C, making it quite warm.

There will be some chance of rain in the early afternoon to evening, with precipitation probabilities increasing to around 53% to 75%, indicating possible showers or thunderstorms during that period. The weather codes suggest a mix of partly cloudy, rain, and thunderstorms mainly in the afternoon and early evening.

Later in the evening and night, the temperatures will cool down again to about 22°C to 20°C with decreased chances of rain, making it a mostly dry night.

So, if you're planning to be outdoors, it’s best to be prepared for warm weather during the day but also some possible rain or storms in the afternoon/evening.